In [1]:
# | default_exp core.agent

In [1]:
# | export
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
import json
from agentic.llms.client import LLMClient
from agentic.configs.prompt_manager import get_system_prompt
from agentic.configs.loader import get_model_config, get_settings_config

In [2]:
# | export
@dataclass
class Message:
    role: str
    content: str
    tool_calls: Optional[List[Dict]] = None
    tool_call_id: Optional[str] = None


@dataclass
class AgentConfig:
    name: str
    instructions: str = ""
    model: Optional[str] = None
    tools: List[str] = field(default_factory=list)
    temperature: float = 0.7
    max_tokens: Optional[int] = None


class Agent:
    """Core Agent class with tool execution and conversation management"""
    
    def __init__(self, config: AgentConfig, llm_client: Optional[LLMClient] = None):
        self.config = config
        self.llm_client = llm_client or self._create_default_llm_client()
        self.conversation_history: List[Message] = []
        self.tools_registry: Dict[str, Callable] = {}
        self.guardrails: List[Callable] = []
        
        # Initialize with system message
        system_prompt = config.instructions or get_system_prompt()
        self.conversation_history.append(Message(role="system", content=system_prompt))
    
    def _create_default_llm_client(self) -> LLMClient:
        """Create default LLM client from config"""
        model_config = get_model_config()
        return LLMClient(
            model=self.config.model or model_config.get('name'),
            base_url=model_config.get('url'),
            api_key=model_config.get('api_key')
        )
    
    def add_tool(self, name: str, tool_func: Callable):
        """Register a tool with the agent"""
        self.tools_registry[name] = tool_func
    
    def add_guardrail(self, guardrail_func: Callable):
        """Add a guardrail function"""
        self.guardrails.append(guardrail_func)
    
    def run(self, message: str, **kwargs) -> Dict[str, Any]:
        """Execute agent with message and return response"""
        # Apply guardrails
        for guardrail in self.guardrails:
            if not guardrail(message):
                return {"content": "Request blocked by guardrails", "blocked": True}
        
        # Add user message
        self.conversation_history.append(Message(role="user", content=message))
        
        # Get available tools
        available_tools = self._get_available_tools()
        
        # Create completion
        messages = self._format_messages_for_llm()
        stream = kwargs.get('stream', True)
        response = self.llm_client.create_completion(
            messages=messages,
            tools=available_tools,
            **kwargs
        )
        
        # Process response based on streaming mode
        if stream:
            result = self.llm_client.handle_streaming_response(response)
        else:
            result = self.llm_client.process_response(response)
        
        # Execute tool calls if any
        if result.get("tool_calls"):
            executed_calls = self._execute_tool_calls(result["tool_calls"])
            result["tool_calls"] = executed_calls
        
        # Add to conversation history
        assistant_message = Message(
            role="assistant", 
            content=result.get("content", ""),
            tool_calls=result.get("tool_calls")
        )
        self.conversation_history.append(assistant_message)
        
        return result
    
    def _get_available_tools(self) -> List[Dict]:
        """Get OpenAI-formatted tools for the configured tool names"""
        from agentic.tools.manager import ToolManager
        tool_manager = ToolManager()
        return tool_manager.get_tools(self.config.tools)
    
    def _execute_tool_calls(self, tool_calls: List[Dict]) -> List[Dict]:
        """Execute tool calls using registered tools"""
        from agentic.tools.display import ToolExecutionDisplay
        display = ToolExecutionDisplay()
        
        executed_calls = []
        
        for tool_call in tool_calls:
            function_name = tool_call["function"]["name"]
            arguments = json.loads(tool_call["function"]["arguments"])
            
            # Show tool execution start
            display.show_tool_start(function_name, trusted=True)
            
            if function_name in self.tools_registry:
                try:
                    result = self.tools_registry[function_name](**arguments)
                    tool_call["result"] = result
                    executed_calls.append(tool_call)
                except Exception as e:
                    tool_call["error"] = str(e)
                    display.show_tool_error(f"Error in {function_name}", str(e))
                    executed_calls.append(tool_call)
            else:
                from ..tools.core import ToolManager
                tool_manager = ToolManager()
                try:
                    result = tool_manager.execute_tool(function_name, arguments)
                    tool_call["result"] = result
                    executed_calls.append(tool_call)
                except Exception as e:
                    tool_call["error"] = str(e)
                    display.show_tool_error(f"Error in {function_name}", str(e))
                    executed_calls.append(tool_call)
        
        return executed_calls
    
    def _format_messages_for_llm(self) -> List[Dict]:
        """Convert Message objects to LLM format"""
        messages = []
        for msg in self.conversation_history:
            message_dict = {"role": msg.role, "content": msg.content}
            if msg.tool_calls:
                message_dict["tool_calls"] = msg.tool_calls
            if msg.tool_call_id:
                message_dict["tool_call_id"] = msg.tool_call_id
            messages.append(message_dict)
        return messages
    
    def clear_history(self):
        """Clear conversation history except system message"""
        system_msg = self.conversation_history[0]
        self.conversation_history = [system_msg]


In [3]:
# Example Usage 
config = AgentConfig(
    name="demo_agent",
    instructions="You are a helpful assistant that can perform calculations.",
    tools=[]
)

agent = Agent(config)

In [4]:

# Add a simple tool
def calculator(operation: str, a: float, b: float) -> dict:
    """Simple calculator tool"""
    if operation == "add":
        result = a + b
    elif operation == "subtract":
        result = a - b
    elif operation == "multiply":
        result = a * b
    elif operation == "divide":
        result = a / b if b != 0 else "Error: Division by zero"
    else:
        result = "Error: Unknown operation"
    
    return {"result": result, "operation": operation, "inputs": [a, b]}

# Register the tool
agent.add_tool("calculator", calculator)

# Test the agent
response = agent.run("Calculate 15 + 25")
print(f"Response: {response.get('content', 'No content')}")
print()

TypeError: BaseTool.__init__() missing 1 required positional argument: 'metadata'